<a href="https://colab.research.google.com/github/GeorgeTelles/modelo_preditivo_ibov_GARCH/blob/main/Previs%C3%A3o_Volatilidade_GARCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
  <img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_clara.png" alt="logo clara" width="300" style="display: inline-block; vertical-align: top; margin-right: 10px;">
  <img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_dark.png" alt="logo dark" width="300" style="display: inline-block; vertical-align: top;">
</div>

# Prevendo a Volatilidade do IBOV

Este projeto tem como objetivo prever a volatilidade do Índice Bovespa (IBOV) utilizando o modelo GARCH (Generalized AutoRegressive Conditional Heteroskedasticity). O modelo GARCH é amplamente utilizado em finanças para modelar e prever a volatilidade dos retornos de ativos financeiros.

## Sobre o Projeto

O IBOV é um dos principais índices de mercado de ações no Brasil e sua volatilidade é uma métrica importante para investidores e analistas financeiros. A volatilidade reflete a variação dos retornos de um ativo e pode fornecer insights valiosos sobre o risco e a dinâmica do mercado.

Para prever essa volatilidade, o projeto utiliza o modelo GARCH, que é uma extensão do modelo ARCH (AutoRegressive Conditional Heteroskedasticity). O GARCH é projetado para modelar séries temporais onde a variância dos erros não é constante ao longo do tempo, mas sim depende das variâncias passadas.

## Funcionalidades

- **Coleta de Dados:** Importação de dados históricos do IBOV para análise. O projeto pode utilizar APIs financeiras ou datasets disponíveis publicamente.
- **Pré-processamento de Dados:** Limpeza e preparação dos dados para a modelagem. Isso inclui a normalização dos retornos e a remoção de outliers.
- **Modelagem GARCH:** Implementação do modelo GARCH para prever a volatilidade. O projeto utiliza a biblioteca `arch` ou outras ferramentas semelhantes em Python.
- **Avaliação do Modelo:** Avaliação da precisão das previsões de volatilidade utilizando métricas adequadas. Comparação dos resultados com dados históricos para validar o modelo.
- **Visualização:** Geração de gráficos e relatórios para visualizar as previsões de volatilidade e analisar os resultados.

## Tecnologias e Bibliotecas Utilizadas

- **Python:** Linguagem de programação principal.
- **pandas e numpy:** Manipulação e análise de dados.
- **arch:** Biblioteca para modelagem GARCH e outras técnicas de volatilidade.
- **matplotlib e seaborn:** Visualização de dados e resultados.


In [ ]:
!pip install arch

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from arch import arch_model
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go

Obtendo dados do IBOV

In [ ]:
ibov = yf.download('^BVSP', start='2018-01-01', end='2024-08-01')
ibov

##Relação entre Retorno e Volatilidade

In [ ]:
#Calculando retornos

ibov['log_retornos'] = np.log(ibov['Adj Close'].pct_change()+1)*100

In [ ]:
ibov['log_retornos'].plot()

In [ ]:
ibov['vol_realizada'] = ibov['log_retornos'].rolling(14).std()*((252)**0.5)
ibov['vol_realizada']

In [ ]:
ibov['vol_realizada'].plot()

#Prevendo a volatilidade de 1 dia

In [ ]:
#Função para prever a volatilidade de 1 dia
def garch_vol(log_retornos):
    # Define model
    garch = arch_model(log_retornos, vol='GARCH', p=1, q=1, dist='skewt')

    # Fit the model
    garch_fit = garch.fit(disp='off')

    # Make forecast
    prev_var = garch_fit.forecast(horizon=1).variance.values[-1]

    # Forecasted volatility
    prev_vol = prev_var**0.5

    # Calculate and return the annualised forecasted variance
    vol_anualizada = prev_vol * (252**0.5)
    return vol_anualizada

In [ ]:
# Calculate the GARCH predicted volatility for the each day
ibov['GARCH_vol_pred'] = ibov['log_retornos'].rolling(252).apply(garch_vol, raw=True)

# Shift the GARCH predicted volatility to match with the actual historical volatility on each day
ibov['GARCH_vol_pred'] = ibov['GARCH_vol_pred'].shift(1)

**Dados com Previsão em tabela**

In [ ]:
ibov

**Ultimas previsões**

In [ ]:
ibov.tail()

#Visualisação das previsão vs a realidade

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(name='Vol Realizada', x=ibov.index, y=ibov['vol_realizada']))
fig.add_trace(go.Scatter(name='GARCH Model', x=ibov.index, y=ibov['GARCH_vol_pred']))
fig.update_layout(title_text='Prevendo Volatilidade IBOV Modelo GARCH', template='plotly_dark')